In [75]:
#import ee
#ee.Authenticate()

In [1]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [2]:
ee.Initialize()

In [3]:
ruta_catastro='projects/ee-bismarksr17/assets/CATASTRO_2023-03-04'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_01_2023'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

In [4]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [85]:
ruta_fases = 'fases.xlsx'

In [86]:
df = pd.read_excel(ruta_fases)

In [93]:
df_select = df[df['fase']==0]
df_select.head(3)

,cod_ca,area,fase
1327,807,2.990119,0
1328,1265,2.988668,0
1329,260,2.946992,0


In [94]:
len(list(df_select['cod_ca']))

97

In [95]:
#lista_props = [852, 1322, 348, 48, 1100, 418, 1148, 1425, 305, 1323, 62, 522, 1216, 1447, 1294, 244, 1065, 1461, 1446, 1615, 484, 1442, 558, 992, 1413, 182, 1199, 1539, 1423, 374, 400, 988, 176, 1049, 408, 1367, 559, 231, 14, 1807, 1009, 369, 1412, 1735, 419, 1092, 271, 534, 507, 290, 572, 1484, 32, 661, 986, 392, 183, 1485, 1151, 391, 1013, 1695, 705, 1483, 1830, 1428, 21, 556, 971, 1097, 628, 439, 288, 110, 1758, 1520, 376, 1241, 1240, 1198, 1282, 903, 195, 1605, 142, 1088, 586, 858, 989, 224, 265, 853, 1522, 115, 1005, 395, 43, 1052, 1474, 1269, 646, 526, 276, 1619, 898, 503, 414, 761, 766, 505, 1025, 523, 859, 1306, 627, 1593, 1430, 1104, 613, 1720, 485, 4, 36, 794, 298, 1731, 1432, 1816, 1656, 189, 173, 990, 947, 1264, 360, 302, 211, 155, 730, 1091, 715, 187, 902, 313, 631, 287, 76, 734, 1697, 1277, 292, 171, 883, 500, 910, 1136, 15, 537, 520, 1260, 1042, 1174]
lista_props = list(df_select['cod_ca'])

#lista_props = [1024]

In [5]:
#ista_props = [1065, 1854, 1826, 1856, 1857, 1858, 1859, 1860, 1861, 1864, 1866, 1865, 592, 1867, 1868]
lista_props = [112, 551, 257, 408, 771, 738, 739, 26, 796, 1010, 1211, 1283, 943, 1443, 1441, 1225, 1467, 1605, 1646, 1642, 1503, 1736, 1739, 737, 1741, 1740, 1260, 1284, 612, 761, 1851, 360, 1833]

In [6]:
len(lista_props)

33

In [7]:
df_intersects = None

In [8]:
contador = 0
for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_geopandas(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    intersect = overlay(lotes_local, vector_local, how="intersection")
    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
    
print('FIN......!')

inicio: 112
fin:  112
___________________CONTADOR:  1
inicio: 551
fin:  551
___________________CONTADOR:  2
inicio: 257
fin:  257
___________________CONTADOR:  3
inicio: 408
fin:  408
___________________CONTADOR:  4
inicio: 771
fin:  771
___________________CONTADOR:  5
inicio: 738
fin:  738
___________________CONTADOR:  6
inicio: 739
fin:  739
___________________CONTADOR:  7
inicio: 26
fin:  26
___________________CONTADOR:  8
inicio: 796
fin:  796
___________________CONTADOR:  9
inicio: 1010
fin:  1010
___________________CONTADOR:  10
inicio: 1211
fin:  1211
___________________CONTADOR:  11
inicio: 1283
fin:  1283
___________________CONTADOR:  12
inicio: 943
fin:  943
___________________CONTADOR:  13
inicio: 1443
fin:  1443
___________________CONTADOR:  14
inicio: 1441
fin:  1441
___________________CONTADOR:  15
inicio: 1225
fin:  1225
___________________CONTADOR:  16
inicio: 1467
fin:  1467
___________________CONTADOR:  17
inicio: 1605
fin:  1605
___________________CONTADOR:  18
inici

In [9]:
len(df_intersects)

25127

In [10]:
df_intersects.to_file("INTERSECT_S1.shp", driver="ESRI Shapefile")

In [32]:
Map = geemap.Map(basemap="SATELLITE")

In [92]:
Map.centerObject(propiedad.geometry(), 14)
Map.addLayer(lotes_canha.style(**vis_params_caña), {}, 'Lotes Caña')
Map.addLayer(lotes_renov.style(**vis_params_renov), {}, 'Lotes Renovación')

In [33]:
Map.centerObject(propiedad.geometry(), 14)
Map.addLayer(lotes_canha.style(**vis_params_caña), {}, 'Lotes Caña')
Map.addLayer(lotes_renov.style(**vis_params_renov), {}, 'Lotes Renovación')

In [34]:
Map

Map(center=[-17.240370201942387, -63.1623575923626], controls=(WidgetControl(options=['position', 'transparent…

In [124]:
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']